<a href="https://colab.research.google.com/github/zfe-dev/PySpark-GoogleColab-Introduction/blob/main/PySpark_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1: Connect to Google Drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
new_retail_data.csv  PySpark_Introduction.ipynb


1.1: If not in the good folder, moves to it

In [ ]:
%cd /content/drive/MyDrive/Github/PySpark_Introduction/


2: Import and setup PySpark

In [11]:
!pip install pyspark findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3b058f38caa5eb8eb0b8cddcc7ec23499e9ba8d60032a8e5cbd2d91ece11d8ad
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [12]:
import findspark
findspark.init()

3: Name the project and let's go!

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark-Introduction").getOrCreate()